#**Expenses and Income Prediction**

##**Assignment**
Given the data set of transactions of 10k users, define regression models for predicting what are the total expenses and the income of a user.


###**Data Description**

The file transactions_data_training.csv contains records of transactions. The other two files, transaction_types.csv and mcc_group_definition.csv are dictionaries providing explanations about values in transaction_type and mcc_group columns from the main dataset respectively.

Each transaction, depending on its type, may represent money flowing either into or from a user's account. Before defining the model, you should therefore extract the target variables: the total expenses and total income of each user based on the combination of amount_n26_currency and transaction_type columns.

###**Practicalities**

Define, train and evaluate predictive models that take as the input the data provided. You may want to split the data into training, testing and validation sets, according to your discretion. Do not use external data for this project. You may use any algorithm of your choice or compare multiple models.

Make sure that the solution reflects your entire thought process - it is more important how the code is structured rather than the final metrics.


#### To download the dataset <a href="https://drive.google.com/drive/folders/1LTOQHJkbaJeIerrhQVca2ev86q3YNCBK?usp=sharing"> Click here </a>

In [9]:
import pandas as pd

# Load the datasets
transactions_df = pd.read_csv('transactions_data_training.csv')
transaction_types_df = pd.read_csv('transaction_types.csv')
mcc_group_definition_df = pd.read_csv('mcc_group_definition.csv')

# Inspect the datasets
print("Transactions DataFrame:")
print(transactions_df.head())
print(transactions_df.info())
print(transactions_df.isnull().sum())

print("\nTransaction Types DataFrame:")
print(transaction_types_df.head())
print(transaction_types_df.info())
print(transaction_types_df.isnull().sum())

print("\nMCC Group Definition DataFrame:")
print(mcc_group_definition_df.head())
print(mcc_group_definition_df.info())
print(mcc_group_definition_df.isnull().sum())


Transactions DataFrame:
                            user_id transaction_date transaction_type  \
0  a78884f5e76951188c1e719d4956773a       2016-02-01               DT   
1  b0333294fef6ff1299102a70ad46b126       2016-02-01               DT   
2  7b8d2a2780adae0cd0c248e92c1b28dc       2016-02-01               DT   
3  cc4abaa500f7db4390ae3f02bd36d805       2016-02-01               DT   
4  49b99d1d5ba028566639e8b3eb7c055b       2016-02-01               DT   

   mcc_group  amount_n26_currency dataset_transaction dataset_user  
0        NaN                  350            training     training  
1        NaN                  202            training     training  
2        NaN                  291            training     training  
3        NaN                  214            training     training  
4        NaN                  272            training     training  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 408546 entries, 0 to 408545
Data columns (total 7 columns):
 #   Column  

In [10]:
# Convert 'transaction_date' to datetime
transactions_df['transaction_date'] = pd.to_datetime(transactions_df['transaction_date'])

# Check for unique transaction types and their directions to understand income vs expenses
print("Unique Transaction Types and Directions:")
print(transaction_types_df[['type', 'direction']].drop_duplicates())

# Handle missing values in 'mcc_group'
# We can either drop these rows or fill them, but for now, let's drop them for simplicity
transactions_df.dropna(subset=['mcc_group'], inplace=True)

# Check the cleaned DataFrame
print("\nCleaned Transactions DataFrame Info:")
print(transactions_df.info())


Unique Transaction Types and Directions:
   type direction
0    AR         -
1    AE        In
2    AV        In
3   BBU        In
4    CT        In
5    DI        In
6    DR        In
7    PF        In
8    PR        In
9    WU        In
10   AA       Out
11  BUB       Out
12   DD       Out
13   DT       Out
14   FT       Out
15   PT       Out
16  TUB       Out
17  TBU        In
18  WEE       Out
19   AL        In
20   AU       Out
21  BUS       Out
22  BRU       Out

Cleaned Transactions DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
Index: 258324 entries, 526 to 408411
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   user_id              258324 non-null  object        
 1   transaction_date     258324 non-null  datetime64[ns]
 2   transaction_type     258324 non-null  object        
 3   mcc_group            258324 non-null  float64       
 4   amount_n26_currency  258324 n

In [11]:
# Create a mapping for transaction types
income_types = transaction_types_df[transaction_types_df['direction'] == 'In']['type'].tolist()
expense_types = transaction_types_df[transaction_types_df['direction'] == 'Out']['type'].tolist()

# Filter for income and expenses
income_df = transactions_df[transactions_df['transaction_type'].isin(income_types)]
expense_df = transactions_df[transactions_df['transaction_type'].isin(expense_types)]

# Aggregate total income and expenses by user_id
total_income = income_df.groupby('user_id')['amount_n26_currency'].sum().reset_index()
total_income.rename(columns={'amount_n26_currency': 'total_income'}, inplace=True)

total_expenses = expense_df.groupby('user_id')['amount_n26_currency'].sum().reset_index()
total_expenses.rename(columns={'amount_n26_currency': 'total_expenses'}, inplace=True)

# Merge total income and expenses into a single DataFrame
user_financials = pd.merge(total_income, total_expenses, on='user_id', how='outer').fillna(0)

# Check the resulting DataFrame
print("\nUser Financials DataFrame:")
print(user_financials.head())



User Financials DataFrame:
   total_income                           user_id  total_expenses
0           0.0  000295594379774ab9ac2c78c946d615             884
1           0.0  000aa792d73dd82b16a29692772d395a             162
2           0.0  0011103fe4ba1264342882b7ab98c641            1313
3           0.0  001679c77c33d7efabf800596fb2a978            3313
4           0.0  001e72a2f4c4376f10011735dde39cd5             600


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Prepare feature and target sets
X = user_financials[['total_expenses']]  # We can use expenses to predict income
y = user_financials['total_income']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")


Mean Squared Error: 0.0


In [13]:
from sklearn.model_selection import train_test_split

# Prepare feature and target sets
X = user_financials[['total_expenses']]
y = user_financials['total_income']

# Split the data into training (60%), validation (20%), and testing (20%) sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Check the sizes of the splits
print(f"Training set size: {X_train.shape[0]}")
print(f"Validation set size: {X_val.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")


Training set size: 5412
Validation set size: 1804
Test set size: 1805


In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Initialize models
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(random_state=42)
}

# Dictionary to hold results
results = {}

# Train and evaluate each model
for model_name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    
    # Validate the model
    y_val_pred = model.predict(X_val)
    mse = mean_squared_error(y_val, y_val_pred)
    
    # Store results
    results[model_name] = mse

# Display results
for model_name, mse in results.items():
    print(f"{model_name}: Mean Squared Error = {mse}")


Linear Regression: Mean Squared Error = 0.0
Decision Tree: Mean Squared Error = 0.0
Random Forest: Mean Squared Error = 0.0


In [15]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation for each model
cv_results = {}

for model_name, model in models.items():
    scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
    mean_mse = -scores.mean()  # Convert negative MSE to positive
    cv_results[model_name] = mean_mse

# Display cross-validation results
for model_name, mean_mse in cv_results.items():
    print(f"{model_name}: Cross-Validated Mean Squared Error = {mean_mse}")


Linear Regression: Cross-Validated Mean Squared Error = -0.0
Decision Tree: Cross-Validated Mean Squared Error = -0.0
Random Forest: Cross-Validated Mean Squared Error = -0.0
